In [1]:
import pulp

In [20]:
import pandas as pd

In [2]:
pulp.listSolvers(onlyAvailable=True)

['CPLEX_PY', 'PULP_CBC_CMD', 'SCIP_PY']

In [3]:
pulp.listSolvers()

['GLPK_CMD',
 'PYGLPK',
 'CPLEX_CMD',
 'CPLEX_PY',
 'GUROBI',
 'GUROBI_CMD',
 'MOSEK',
 'XPRESS',
 'XPRESS',
 'XPRESS_PY',
 'PULP_CBC_CMD',
 'COIN_CMD',
 'COINMP_DLL',
 'CHOCO_CMD',
 'MIPCL_CMD',
 'SCIP_CMD',
 'FSCIP_CMD',
 'SCIP_PY',
 'HiGHS',
 'HiGHS_CMD',
 'COPT',
 'COPT_DLL',
 'COPT_CMD',
 'SAS94',
 'SASCAS']

In [4]:
from utils.utils import read_jsonl

In [16]:
from evaluation.metrics import (
    get_spans_from_binary_labels,
    disorder_of_a_unitary_alignment,
    disorder_of_an_alignment,
    dissimilarity_of_span_pair,
    get_observed_disorder,
    get_auprc,
    get_r_precision
)
from sklearn.metrics import (
    f1_score,
    precision_score,
    recall_score,
    accuracy_score
)

In [6]:
expert_set = read_jsonl('./annotation/annotated_result/all/setting2/expert.jsonl')
test_set = read_jsonl('./annotation/annotated_result/all/setting2/test.jsonl')

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1]

In [100]:
# test_id = '20200211_10-K_1585689_part2_item7_para3' # simple
# test_id = '20200220_10-K_1045810_part2_item7_para45' # long
# test_id = '20220222_10-K_1090727_part2_item7_para153' # no signal
# test_id = '20200220_10-K_1045810_part2_item7_para52' # short and good naive
test_id = '20220222_10-K_1090727_part2_item7_para71'
for e in expert_set:
    if test_id in e.keys():
        reference = e[test_id]['binary_labels']
for t in test_set:
    if t['sample_id'] == test_id:
        prediction = t['naive_aggregation']['label']
prediction2 = [0] * len(prediction)
# prediction3 = [1, 0] * int(len(prediction)/2)
prediction3 = [1] * len(prediction)
prediction4 = [1] + prediction[1:]
prediction5 = [1] + reference[1:]
prediction6 = reference[1:] + [1]
preds = [reference, prediction5, prediction, prediction4, prediction2, prediction3, prediction6]
names = ['truth', 'truth+1', 'naive', 'naive+1', 'all_0', 'all_1', 'truth+1tail']
metrics = []
for pred in preds:
    metrics.append({
        'disorder': get_observed_disorder(reference, pred),
        'f1': f1_score(reference, pred),
        'auprc': get_auprc(reference, pred),
        # 'auprc', get_auprc(reference, pred),
        'r_prec': get_r_precision(reference, pred),
        # 'accuracy': accuracy_score(reference, pred),
        # 'precision': precision_score(reference, pred),
        # 'recall': recall_score(reference, pred)
        
    })
pd.DataFrame(metrics, index=names)

disorder        f1     auprc    r_prec
truth        0.000000  1.000000  1.000000  1.000000
truth+1      0.666667  0.947368  0.950000  0.900000
naive        0.033058  0.818182  0.846154  0.692308
naive+1      0.688705  0.782609  0.821429  0.642857
all_0        1.000000  0.000000  0.666667  0.000000
all_1        0.250000  0.500000  0.666667  0.333333
truth+1tail  0.002770  0.947368  0.950000  0.900000

In [94]:
for e in expert_set:
    if test_id in e.keys():
        print(e[test_id]['tokens'])
        print(e[test_id]['binary_labels'])
for t in test_set:
    if t['sample_id'] == test_id:
        print(t['naive_aggregation']['label'])
        print(t['loose_aggregation']['label'])
        print(t['strict_aggregation']['label'])

['United', 'parcel', 'service,', 'inc.', 'and', 'subsidiaries', "Management's", 'discussion', 'and', 'analysis', 'of', 'financial', 'condition', 'and', 'Results', 'of', 'operations', 'Revenue', 'total', 'revenue', 'for', 'supply', 'chain', 'solutions', 'increased', '$2.2', 'billion.']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [29]:
import pandas as pd
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

# 載入 expert.ids 裡的 test_id
with open('./annotation/annotated_result/all/setting2/expert.ids') as f:
    test_ids = [line.strip() for line in f if line.strip()]

# 存放所有結果
all_results = []

for test_id in test_ids:
    reference = None
    prediction = None

    # 找到對應 reference
    for e in expert_set:
        if test_id in e.keys():
            reference = e[test_id]['binary_labels']
            break

    # 找到對應 prediction
    for t in test_set:
        if t['sample_id'] == test_id:
            prediction = t['naive_aggregation']['label']
            break

    # 若找不到就跳過
    if reference is None or prediction is None:
        print(f"找不到對應資料：{test_id}")
        continue

    prediction2 = [0] * len(prediction)
    prediction3 = [1] * len(prediction)
    prediction4 = [1] + prediction[1:]
    prediction5 = [1] + reference[1:]
    
    preds = [reference, prediction5, prediction, prediction4, prediction2, prediction3]
    names = ['truth', 'truth+1', 'naive', 'naive+1', 'all_0', 'all_1']

    for name, pred in zip(names, preds):
        all_results.append({
            'test_id': test_id,
            'variant': name,
            'disorder': get_observed_disorder(reference, pred),
            'f1': f1_score(reference, pred),
            'r_prec': get_r_precision(reference, pred),
            'accuracy': accuracy_score(reference, pred),
            'precision': precision_score(reference, pred),
            'recall': recall_score(reference, pred)
        })
        if name == 'truth' and f1_score(reference, pred) != 1:
            print(reference)
            print('-'*50)

# 轉成 dataframe 顯示或輸出
df = pd.DataFrame(all_results)
# df.to_csv('evaluation_result.csv', index=False)  # 可選：存成 CSV
df

/tmp2/yshuang/fin.rag/fintext_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/tmp2/yshuang/fin.rag/fintext_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/tmp2/yshuang/fin.rag/fintext_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/tmp2

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
--------------------------------------------------
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
--------------------------------------------------


/tmp2/yshuang/fin.rag/fintext_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/tmp2/yshuang/fin.rag/fintext_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/tmp2/yshuang/fin.rag/fintext_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/tmp2

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
--------------------------------------------------


/tmp2/yshuang/fin.rag/fintext_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/tmp2/yshuang/fin.rag/fintext_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/tmp2/yshuang/fin.rag/fintext_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/tmp2

...

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
--------------------------------------------------


/tmp2/yshuang/fin.rag/fintext_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/tmp2/yshuang/fin.rag/fintext_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/tmp2/yshuang/fin.rag/fintext_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/tmp2/yshuang/fin.rag/fintex

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
--------------------------------------------------
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
--------------------------------------------------


/tmp2/yshuang/fin.rag/fintext_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/tmp2/yshuang/fin.rag/fintext_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/tmp2/yshuang/fin.rag/fintext_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/tmp2

test_id  variant  disorder   f1  r_prec  \
0    20200211_10-K_1585689_part2_item7_para15    truth       0.0  0.0     0.0   
1    20200211_10-K_1585689_part2_item7_para15  truth+1       1.0  0.0     0.0   
2    20200211_10-K_1585689_part2_item7_para15    naive       0.0  0.0     0.0   
3    20200211_10-K_1585689_part2_item7_para15  naive+1       1.0  0.0     0.0   
4    20200211_10-K_1585689_part2_item7_para15    all_0       0.0  0.0     0.0   
..                                        ...      ...       ...  ...     ...   
595   20221028_10-K_320193_part2_item7_para44  truth+1       1.0  0.0     0.0   
596   20221028_10-K_320193_part2_item7_para44    naive       0.0  0.0     0.0   
597   20221028_10-K_320193_part2_item7_para44  naive+1       1.0  0.0     0.0   
598   20221028_10-K_320193_part2_item7_para44    all_0       0.0  0.0     0.0   
599   20221028_10-K_320193_part2_item7_para44    all_1       1.0  0.0     0.0   

     accuracy  precision  recall  
0    1.000000        0.0     0.0  
1    0.994286        0.0     0.0  
2    1.000000        0.0     0.0  
3    0.994286        0.0     0.0  
4    1.000000        0.0     0.0  
..        ...        ...     ...  
595  0.987654        0.0     0.0  
596  1.000000        0.0     0.0  
597  0.987654        0.0     0.0  
598  1.000000        0.0     0.0  
599  0.000000        0.0     0.0  

[600 rows x 8 columns]

In [27]:
summary = df.groupby('variant').agg({
    'disorder': ['mean', 'std', 'min', 'max'],
    'f1': ['mean', 'std', 'min', 'max'],
    'r_prec': ['mean', 'std', 'min', 'max'],
    'accuracy': ['mean', 'std', 'min', 'max'],
    'precision': ['mean', 'std', 'min', 'max'],
    'recall': ['mean', 'std', 'min', 'max']
})

# 排版更漂亮一點：把多層 column 攤平
summary.columns = ['_'.join(col) for col in summary.columns]
summary.reset_index(inplace=True)

# 顯示出來
summary

variant  disorder_mean  disorder_std  disorder_min  disorder_max   f1_mean  \
0    all_0       0.463381      0.406657           0.0           1.0  0.000000   
1    all_1       0.562633      0.379001           0.0           1.0  0.349310   
2    naive       0.508642      0.441804           0.0           2.0  0.371713   
3  naive+1       0.597275      0.350492           0.0           2.0  0.367679   
4    truth       0.000000      0.000000           0.0           0.0  0.640000   
5  truth+1       0.518136      0.428886           0.0           1.0  0.621536   

     f1_std  f1_min  f1_max  r_prec_mean  ...  accuracy_min  accuracy_max  \
0  0.000000     0.0     0.0     0.000000  ...      0.000000           1.0   
1  0.341810     0.0     1.0     0.271164  ...      0.000000           1.0   
2  0.377520     0.0     1.0     0.533521  ...      0.256410           1.0   
3  0.373386     0.0     1.0     0.524588  ...      0.282051           1.0   
4  0.482418     0.0     1.0     0.640000  ...      1.000000           1.0   
5  0.471464     0.0     1.0     0.608252  ...      0.933333           1.0   

   precision_mean  precision_std  precision_min  precision_max  recall_mean  \
0        0.000000       0.000000            0.0            0.0     0.000000   
1        0.271164       0.295269            0.0            1.0     0.640000   
2        0.442191       0.420529            0.0            1.0     0.404409   
3        0.418542       0.399606            0.0            1.0     0.406712   
4        0.640000       0.482418            0.0            1.0     0.640000   
5        0.608252       0.465732            0.0            1.0     0.640000   

   recall_std  recall_min  recall_max  
0    0.000000         0.0         0.0  
1    0.482418         0.0         1.0  
2    0.426005         0.0         1.0  
3    0.426080         0.0         1.0  
4    0.482418         0.0         1.0  
5    0.482418         0.0         1.0  

[6 rows x 25 columns]

In [30]:
# 只看每種 variant 的平均值
mean_summary = df.groupby('variant')[['disorder', 'f1', 'r_prec', 'accuracy', 'precision', 'recall']].mean()

# 顯示出來
mean_summary

disorder        f1    r_prec  accuracy  precision    recall
variant                                                             
all_0    0.463381  0.000000  0.000000  0.728836   0.000000  0.000000
all_1    0.562633  0.349310  0.271164  0.271164   0.271164  0.640000
naive    0.508642  0.371713  0.533521  0.807887   0.442191  0.404409
naive+1  0.597275  0.367679  0.524588  0.799355   0.418542  0.406712
truth    0.000000  0.640000  0.640000  1.000000   0.640000  0.640000
truth+1  0.518136  0.621536  0.608252  0.989387   0.608252  0.640000

In [10]:
print(reference)
print(prediction)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [11]:
for 

0.17161722613004554

In [12]:
get_observed_disorder(reference, reference)

0.0

In [13]:
get_observed_disorder(reference, prediction4)

0.3076923076923077

In [14]:
get_observed_disorder(reference, prediction)

0.25800034814233447

In [ ]:
get_observed_disorder(reference, prediction)

In [38]:
pred_spans, _ = get_spans_from_binary_labels(prediction)
ref_spans, _  = get_spans_from_binary_labels(reference)

In [39]:
ref_spans

[(24, 29),
 (35, 45),
 (46, 47),
 (50, 51),
 (99, 100),
 (139, 154),
 (155, 167),
 (175, 185),
 (186, 209)]

In [50]:
spans_pool = [(s, 1) for s in pred_spans] + [(s, 2) for s in ref_spans]
max_size = len(spans_pool)
max_size_per_ua = len(spans_pool)
possible_unitary_alignments = [tuple(c) for c in pulp.allcombinations(spans_pool, max_size_per_ua) if len(tuple(c)) < 10]

In [51]:
len(possible_unitary_alignments)

4016

In [42]:
len(possible_unitary_alignments)

4095

In [31]:
spans_pool = [(s, 1) for s in pred_spans] + [(s, 2) for s in ref_spans]
max_size = len(spans_pool)
max_size_per_ua = len(spans_pool)
avarage_num_of_span = len(spans_pool) / 2
all_unitary_alignments = [tuple(c) for c in pulp.allcombinations(spans_pool, max_size_per_ua)]
possible_unitary_alignments, disorder_of_possible_ua = [], []
for ua in all_unitary_alignments:
    disorder = disorder_of_a_unitary_alignment(ua)
    if disorder <= 1:
        possible_unitary_alignments.append(ua)
        disorder_of_possible_ua.append(disorder)
ua2i_map = {c: i for i, c in enumerate(possible_unitary_alignments)}
i2ua_map = {v: k for k, v in ua2i_map.items()}
i2ua_list = list(i2ua_map.keys())
x = pulp.LpVariable.dicts(
    "ua", i2ua_map.keys(), lowBound=0, upBound=1, cat=pulp.LpInteger
) # unitary_alignment # name too long error
alignment_disorder_model = pulp.LpProblem("alignment_disorder", pulp.LpMinimize) # alignment_disorder
alignment_disorder_model += pulp.lpSum([disorder_of_possible_ua[i] * x[i] for i in i2ua_list])
alignment_disorder_model += (
    pulp.lpSum([x[i] for i in i2ua_list]) <= max_size,
    "Max_ua", # maxium_number_of_ua
)
for span in spans_pool:
    alignment_disorder_model += (
        pulp.lpSum([x[i] for i in i2ua_list if span in i2ua_map[i]]) == 1,
        f"Must_seat_{span}",
    )

NameError: name 'pred_spans' is not defined

In [ ]:
solver = pulp.FSCIP_CMD('/tmp2/yshuang/fin.rag/scip/bin/fscip', msg=False, threads=32)

In [ ]:
alignment_disorder_model.solve(solver)

In [ ]:
alignment = []
for i in i2ua_list:
    if x[i].value() == 1.:
        print(i2ua_map[i])
        print(disorder_of_a_unitary_alignment(i2ua_map[i]))
        alignment.append(i2ua_map[i])

print(disorder_of_an_alignment(alignment,  avarage_num_of_span))

In [ ]:
pulp.value(alignment_disorder_model.objective)